In [3]:
import numpy as np
import pandas as pd

In [4]:
train = pd.read_csv('../data/train.csv')
gas = pd.read_csv('../data/gas_prices.csv')
electricity = pd.read_csv('../data/electricity_prices.csv')
hw = pd.read_csv('../data/historical_weather.csv')
fw = pd.read_csv('../data/forecast_weather.csv')
wtm = pd.read_csv('../data/weather_station_to_county_mapping.csv')
client = pd.read_csv('../data/client.csv')

## Data featuring

In [ ]:
print(np.round(wtm.isna().sum().sum() / (wtm.shape[0] * wtm.shape[1]) * 100, 2), '% of the',
      wtm.shape[0] * wtm.shape[1], 'weather county mapping values are null.')

In [18]:
wtm['county'].nunique()

16

#### Historical Weather

In [6]:
grouped_hweather = hw.groupby(['datetime']).mean()
merge_temp = grouped_hweather.drop(columns=['data_block_id'])
train = train.merge(merge_temp, how="left", on="datetime")

#### Forecast Weather

In [7]:
grouped_fweather = fw.drop(columns=['origin_datetime'])
grouped_fweather.rename(columns={'forecast_datetime': 'datetime'}, inplace=True)
grouped_fweather = grouped_fweather.groupby(['datetime']).mean()

merge_fweather = grouped_fweather.drop(columns=['latitude', 'longitude', 'data_block_id'])
merge_fweather.columns = list(map(lambda c: "fc_%s" % c, merge_fweather.columns))

train = train.merge(merge_fweather, how="left", on="datetime")

#### Electricity Prices

In [8]:
merge_elec = electricity.filter(items=['euros_per_mwh', 'forecast_date'], axis=1)
merge_elec.rename(columns={"forecast_date": "datetime", "euros_per_mwh": "elec_price"}, inplace=True)

train = train.merge(merge_elec, how="left", on="datetime")

#### Gas Prices

In [9]:
train["forecast_date"] = pd.to_datetime(train["datetime"]).dt.strftime('%Y-%m-%d')

In [10]:
merge_gas = gas.filter(items=['lowest_price_per_mwh', 'highest_price_per_mwh', 'forecast_date'], axis=1)
merge_gas.rename(columns={"lowest_price_per_mwh": "lowest_gas_price", "highest_price_per_mwh": "highest_gas_price"},
                 inplace=True)

train = train.merge(merge_gas, how="left", on="forecast_date")

#### Client

In [11]:
grouped_client = client.drop(columns=['data_block_id', 'is_business'])
grouped_client.rename(columns={'date': 'forecast_date'}, inplace=True)
merge_client = grouped_client.groupby(['forecast_date', 'product_type']).mean()

train = train.merge(merge_client, how="left", on=["forecast_date", "product_type"], suffixes=('', '_clt'))

In [12]:
train.drop(columns='forecast_date', inplace=True)

#### Date formats

In [13]:
import numpy as np

train['hour'] = pd.to_datetime(train['datetime']).dt.hour
train['sin_hour'] = np.sin(2 * np.pi * train['hour'] / max(train['hour']))
train['cos_hour'] = np.cos(2 * np.pi * train['hour'] / max(train['hour']))

train['day'] = pd.to_datetime(train['datetime']).dt.day
train['sin_day'] = np.sin(2 * np.pi * train['day'] / max(train['day']))
train['cos_day'] = np.cos(2 * np.pi * train['day'] / max(train['day']))

train['month'] = pd.to_datetime(train['datetime']).dt.month
train['sin_month'] = np.sin(2 * np.pi * train['month'] / max(train['month']))
train['cos_month'] = np.cos(2 * np.pi * train['month'] / max(train['month']))

train['weekday'] = pd.to_datetime(train['datetime']).dt.weekday
train['year'] = pd.to_datetime(train['datetime']).dt.year

train.drop(columns=['datetime', 'row_id', 'hour', 'day', 'month'], inplace=True)

In [14]:
train

,county,is_business,product_type,target,is_consumption,data_block_id,prediction_unit_id,temperature,dewpoint,rain,...,eic_count,installed_capacity,sin_hour,cos_hour,sin_day,cos_day,sin_month,cos_month,weekday,year
0,0,0,1,0.713,0,0,0,13.308929,10.9875,0.013393,...,20.863636,360.281364,0.000000e+00,1.0,2.012985e-01,0.97953,-1.0,-1.836970e-16,2,2021
1,0,0,1,96.590,1,0,0,13.308929,10.9875,0.013393,...,20.863636,360.281364,0.000000e+00,1.0,2.012985e-01,0.97953,-1.0,-1.836970e-16,2,2021
2,0,0,2,0.000,0,0,1,13.308929,10.9875,0.013393,...,8.600000,110.180000,0.000000e+00,1.0,2.012985e-01,0.97953,-1.0,-1.836970e-16,2,2021
3,0,0,2,17.314,1,0,1,13.308929,10.9875,0.013393,...,8.600000,110.180000,0.000000e+00,1.0,2.012985e-01,0.97953,-1.0,-1.836970e-16,2,2021
4,0,0,3,2.904,0,0,2,13.308929,10.9875,0.013393,...,87.433333,1924.952333,0.000000e+00,1.0,2.012985e-01,0.97953,-1.0,-1.836970e-16,2,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,637,64,NaN,NaN,NaN,...,NaN,NaN,-2.449294e-16,1.0,-2.449294e-16,1.00000,0.5,-8.660254e-01,2,2023
2018348,15,1,1,0.000,0,637,59,NaN,NaN,NaN,...,NaN,NaN,-2.449294e-16,1.0,-2.449294e-16,1.00000,0.5,-8.660254e-01,2,2023
2018349,15,1,1,28.404,1,637,59,NaN,NaN,NaN,...,NaN,NaN,-2.449294e-16,1.0,-2.449294e-16,1.00000,0.5,-8.660254e-01,2,2023
2018350,15,1,3,0.000,0,637,60,NaN,NaN,NaN,...,NaN,NaN,-2.449294e-16,1.0,-2.449294e-16,1.00000,0.5,-8.660254e-01,2,2023


In [15]:
train.isna().sum().sum()

114050

## Export

In [16]:
# train.to_csv("dataset.csv", index=False)

KeyboardInterrupt: 